In [1]:
import resnet20
import adder

In [2]:
cd Desktop

/home/mohamad.kassab/Desktop


In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
from resnet20 import resnet20
import torch
from torch.autograd import Variable
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 
import argparse
import math
from skimage import io
import pandas as pd
from torchvision.io import read_image


In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


In [5]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

data_set = CustomImageDataset(annotations_file = 'dataset3.csv', img_dir = 'data_dir', transform = transform)


data_train, data_test = torch.utils.data.random_split(data_set,[2000,456])


In [ ]:
#Copyright (C) 2019. Huawei Technologies Co., Ltd. All rights reserved.

#This program is free software; you can redistribute it and/or modify it under the terms of the BSD 3-Clause License.

#This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the BSD 3-Clause License for more details.


parser = argparse.ArgumentParser(description='train-addernet')

# Basic model parameters.

parser.add_argument('output_dir', type=str, default='/cache/models/')
args = parser.parse_args(['/home/mohamad.kassab/Desktop'])

os.makedirs(args.output_dir, exist_ok=True)  

acc = 0
acc_best = 0

data_train_loader = DataLoader(data_train, batch_size=32, shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test, batch_size=5, num_workers=0)

net = resnet20().cuda()
criterion = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    lr = 0.05 * (1+math.cos(float(epoch)/400*math.pi))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
def train(epoch):
    adjust_learning_rate(optimizer, epoch)
    global cur_batch_win
    net.train()
    loss_list, batch_list = [], []
    for i, (images, labels) in enumerate(data_train_loader):
        images, labels = Variable(images).cuda(), Variable(labels).cuda()
 
        optimizer.zero_grad()
 
        output = net(images)
 
        loss = criterion(output, labels)
 
        loss_list.append(loss.data.item())
        batch_list.append(i+1)
 
        if i == 1:
            print('Train - Epoch %d, Batch: %d, Loss: %f' % (epoch, i, loss.data.item()))
 
        loss.backward()
        optimizer.step()
 
 
def test():
    global acc, acc_best
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    with torch.no_grad():
        for i, (images, labels) in enumerate(data_test_loader):
            images, labels = Variable(images).cuda(), Variable(labels).cuda()
            output = net(images)
            avg_loss += criterion(output, labels).sum()
            pred = output.data.max(1)[1]
            total_correct += pred.eq(labels.data.view_as(pred)).sum()
 
    avg_loss /= len(data_test)
    acc = float(total_correct) / len(data_test)
    if acc_best < acc:
        acc_best = acc
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.data.item(), acc))
 
 
def train_and_test(epoch):
    train(epoch)
    test()
 
 
def main():
    epoch = 50
    for e in range(1, epoch):
        train_and_test(e)
    torch.save(net,args.output_dir + 'addernet')
 
 
if __name__ == '__main__':
    main()


Train - Epoch 1, Batch: 1, Loss: 8.119944
Test Avg. Loss: 0.938251, Accuracy: 0.504386
Train - Epoch 2, Batch: 1, Loss: 0.600813
Test Avg. Loss: 0.146088, Accuracy: 0.537281
Train - Epoch 3, Batch: 1, Loss: 0.274956
Test Avg. Loss: 0.108827, Accuracy: 0.883772
Train - Epoch 4, Batch: 1, Loss: 0.127260
Test Avg. Loss: 0.080611, Accuracy: 0.953947
Train - Epoch 5, Batch: 1, Loss: 0.234333
Test Avg. Loss: 0.624193, Accuracy: 0.866228
Train - Epoch 6, Batch: 1, Loss: 0.123369
Test Avg. Loss: 0.050425, Accuracy: 0.971491
Train - Epoch 7, Batch: 1, Loss: 0.068195
Test Avg. Loss: 0.083471, Accuracy: 0.938596
Train - Epoch 8, Batch: 1, Loss: 0.146113
Test Avg. Loss: 0.102507, Accuracy: 0.896930
Train - Epoch 9, Batch: 1, Loss: 0.243558
Test Avg. Loss: 0.051471, Accuracy: 0.932018
Train - Epoch 10, Batch: 1, Loss: 0.163716
Test Avg. Loss: 0.042719, Accuracy: 0.958333
Train - Epoch 11, Batch: 1, Loss: 0.238487
Test Avg. Loss: 0.121432, Accuracy: 0.868421
Train - Epoch 12, Batch: 1, Loss: 0.11580